# 使用HyDE改进文档索引
本笔记介绍了如何使用假设性文档嵌入（HyDE），如[本文](https://arxiv.org/abs/2212.10496)所述。

在高层次上，HyDE是一种嵌入技术，它接受查询，生成假设答案，然后嵌入生成的文档并将其用作最终示例。

为了使用HyDE，因此我们需要提供一个基本嵌入模型，以及一个可以用来生成这些文档的LLMChain。默认情况下，HyDE类带有一些默认提示（有关详细信息，请参阅论文），但我们也可以创建自己的提示。

In [1]:
# 导入所需的模块和类
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings

In [2]:
# 导入OpenAIEmbeddings类
base_embeddings = OpenAIEmbeddings()
# 创建OpenAI类的实例
llm = OpenAI()

In [3]:
# 使用 `web_search` 提示加载
embeddings = HypotheticalDocumentEmbedder.from_llm(llm, base_embeddings, "web_search")

In [4]:
# 现在我们可以将其作为任何嵌入类使用！
# 调用embeddings对象的embed_query方法，传入查询字符串"Where is the Taj Mahal?"作为参数
result = embeddings.embed_query("Where is the Taj Mahal?")

## 多代
我们还可以生成多个文档，然后将这些文档的嵌入进行合并。默认情况下，我们通过取平均值来合并这些文档。我们可以通过改变用于生成文档的LLM来返回多个内容来实现这一点。

In [5]:
# 创建一个OpenAI对象，参数n表示每个动作的候选项数，best_of表示每次选择最佳动作的次数
multi_llm = OpenAI(n=4, best_of=4)

In [6]:
# 使用from_llm方法创建HypotheticalDocumentEmbedder对象，并传入参数multi_llm, base_embeddings, "web_search"
embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, base_embeddings, "web_search"
)

In [7]:
# 调用embeddings库中的embed_query函数，并传入查询语句"Where is the Taj Mahal?"
result = embeddings.embed_query("Where is the Taj Mahal?")

## 使用自定义提示
除了使用预配置的提示之外，我们还可以轻松构建自己的提示，并在生成文档的LLMChain中使用这些提示。如果我们知道查询所在的领域，这样可以有助于我们将提示条件化，以生成更类似于该领域的文本。

在下面的示例中，让我们将其条件化为生成关于国情咨文的文本（因为我们将在下一个示例中使用它）。

In [8]:
# 定义一个模板字符串，用于提示用户回答关于最近一次国情咨文的问题
prompt_template = """Please answer the user's question about the most recent state of the union address
Question: {question}
Answer:"""

# 创建一个 PromptTemplate 对象，指定输入变量为 "question"，模板为上述定义的模板字符串
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

# 创建一个 LLMChain 对象，指定语言模型为 llm，提示为上述定义的 PromptTemplate 对象
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [9]:

# 创建一个HypotheticalDocumentEmbedder对象，并传入参数llm_chain和base_embeddings
embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=base_embeddings)

In [10]:

# 调用embeddings库中的embed_query函数，并传入待查询的问题作为参数
result = embeddings.embed_query(
    "What did the president say about Ketanji Brown Jackson"
)

## 使用HyDE
现在我们有了HyDE，我们可以像使用其他嵌入类一样使用它！下面是使用它在国情咨文示例中查找相似段落的示例。

In [11]:
from langchain_community.vectorstores import Chroma  # 导入Chroma向量存储模块
from langchain_text_splitters import CharacterTextSplitter  # 导入字符文本分割模块

with open("../../state_of_the_union.txt") as f:  # 打开文件state_of_the_union.txt
    state_of_the_union = f.read()  # 读取文件内容并赋值给state_of_the_union变量
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # 使用字符文本分割模块创建文本分割器，设置分割大小为1000，重叠为0
texts = text_splitter.split_text(state_of_the_union)  # 使用文本分割器对state_of_the_union进行分割，并将结果赋值给texts变量

In [12]:
# 创建一个Chroma对象，使用给定的文本和嵌入
docsearch = Chroma.from_texts(texts, embeddings)

# 定义查询字符串
query = "What did the president say about Ketanji Brown Jackson"

# 使用相似性搜索来查找与查询字符串相关的文档
docs = docsearch.similarity_search(query)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [13]:
# 打印第一个文档的页面内容
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
